# Euler's Method

This notebook accompanies the [Forward Euler](../odes/forward-euler.md) lecture notes.
We implement forward and backward Euler methods, compare them against exact solutions,
and explore convergence, stability, and stiffness interactively.

In [ ]:
import numpy as np
import scipy.linalg as LA
import matplotlib.pyplot as plt

# Forward Euler

We solve the scalar test problem $u' = au$, $u(0) = 1$, whose exact solution is $u(t) = e^{at}$. Forward Euler with step size $k$ gives $u_{n+1} = u_n + k\,f(t_n, u_n)$. The plot compares the numerical and exact solutions.

In [ ]:
a = 2.0

def f(t, u):
    return a * u

# Initial condition
u0 = 1.0

# Time setup
Tf = 1.25
k  = 1./16  # Time step
numsteps = np.ceil(Tf / k).astype(int)
k = Tf / numsteps 

# Create output vector
vs = np.empty(numsteps+1, dtype=float)
vs[0] = u0

v = u0 # This always holds the current ODE value
for i in range(numsteps):
    t = k * (i - 1)
    v = v + k * f(t, v)
    vs[i+1] = v

# Get timesteps 
tf = np.linspace(0, Tf, 1000)
fig, ax = plt.subplots(1, 1)
plt.plot(tf, np.exp(a * tf), lw=3, label='Exact solution')

# for i in range(1, numsteps+1):
#     C = vs[i] * np.exp(-a * ts[i])
#     plt.plot(tf, C * np.exp(a * tf), alpha=0.75)

ts = np.linspace(0, Tf, numsteps+1)
plt.plot(ts, vs, color='k', lw=2, zorder=5, marker='o', label='Euler\'s methods')

plt.title('Euler\'s method solution k = {0:.4f}'.format(k))
plt.ylabel('$y(t)$')
plt.xlabel('Time')
plt.legend(loc='best')
#plt.ylim([0, 100])
plt.xlim([0, Tf])

# Backward Euler (Secant Method)

Backward Euler is implicit: $u_{n+1} = u_n + k\,f(t_{n+1}, u_{n+1})$. At each step we must solve a nonlinear equation for $u_{n+1}$. Here we use the **secant method** as the nonlinear solver. We apply it to the same test problem $u' = au$ and compare with the exact solution.

In [ ]:
a = 2.0

def f(t, u):
    return a * u

# Initial condition
u0 = 1.0

# Time setup
Tf = 1.25
k  = 1./16  # Time step
numsteps = np.ceil(Tf / k).astype(int)
k = Tf / numsteps 

# Create output vector
vs = np.empty(numsteps+1, dtype=float)
vs[0] = u0

rtol = 1e-4

v = u0 # This always holds the current ODE value
for i in range(numsteps):
    t = k * i  # Time at which we are trying to compute solution at.
    
    # Setup secant method
    v0 = v
    v1 = v + k * f(t, v)
    
    # The function we are trying to find the root of.
    def h(v_old, v_new):
        return v_old + k * f(t, v_new) - v_new
    
    # Secant method
    done = False
    while not done:
        v2 = v1 - h(v, v1) * (v1 - v0) / (h(v, v1) - h(v, v0))
        v0, v1 = v1, v2
        done = abs(v0 - v1) / abs(v0) < rtol

    # Update the solution
    v = v1
    vs[i+1] = v

# Get timesteps 
tf = np.linspace(0, Tf, 1000)
fig, ax = plt.subplots(1, 1)
plt.plot(tf, np.exp(a * tf), lw=3, label='Exact solution')

# for i in range(1, numsteps+1):
#     C = vs[i] * np.exp(-a * ts[i])
#     plt.plot(tf, C * np.exp(a * tf), alpha=0.75)

ts = np.linspace(0, Tf, numsteps+1)
plt.plot(ts, vs, color='k', lw=2, zorder=5, marker='o', label='Euler\'s methods')

plt.title('Euler\'s method solution k = {0:.4f}'.format(k))
plt.ylabel('$y(t)$')
plt.xlabel('Time')
plt.legend(loc='best')
#plt.ylim([0, 100])
plt.xlim([0, Tf])

## Backward Euler (Simplified Newton)

Same problem, but now using a **simplified Newton** iteration as the nonlinear solver. The Jacobian is approximated by finite differences and held fixed across Newton iterates, which is cheaper per iteration than the secant method above.

In [ ]:
from math import sqrt

# Backward Euler with simplified Newton iteration
a = 2.0

def f(t, u):
    return a * u

# Initial condition
u0 = 1.0

# Time setup
Tf = 1.25
k  = 1./16  # Time step
numsteps = np.ceil(Tf / k).astype(int)
k = Tf / numsteps 

# Create output vector
vs = np.empty(numsteps+1, dtype=float)
vs[0] = u0

chi = 1e-1
rtol = 1e-4
atol = 1e-4
delta = sqrt(np.finfo(float).eps)

v = u0 # This always holds the current ODE value
for i in range(numsteps):
    t = k * i  # Time at which we are trying to compute solution at.

    # Get derivative approximation
    df = (f(t, v + delta) - f(t, v)) / delta

    # Setup simplified Newton
    v_new = v
    dv = 1.0
    done = False
    while not done:
        dv_new = (v + k * f(t + k, v_new) - v_new) / (1.0 - k * df)
        theta = abs(dv_new) / abs(dv)
        done = dv_new == 0.0 or abs(dv_new) <= chi * atol * (1 - theta) / theta
        
        dv = dv_new
        v_new += dv_new

    # Update the solution
    v = v_new
    vs[i+1] = v

# Get timesteps 
tf = np.linspace(0, Tf, 1000)
fig, ax = plt.subplots(1, 1)
plt.plot(tf, np.exp(a * tf), lw=3, label='Exact solution')

ts = np.linspace(0, Tf, numsteps+1)
plt.plot(ts, vs, color='k', lw=2, zorder=5, marker='o', label="Backward Euler (Newton)")

plt.title('Backward Euler (simplified Newton) k = {0:.4f}'.format(k))
plt.ylabel('$y(t)$')
plt.xlabel('Time')
plt.legend(loc='best')
plt.xlim([0, Tf])

# Backward Euler with Richardson Adaptive Step Size

We add **adaptive step-size control** to backward Euler using Richardson extrapolation.

**Setup.** Starting from $(t, v)$, backward Euler (order $p=1$) is applied in two ways:

1. **One full step** of size $k$ → $v_{\text{full}}$
2. **Two half-steps** of size $k/2$ → $v_{\text{half}}$

Both approximate $u(t+k)$, but with different leading error terms. For a method of order $p$, the local error after one step of size $h$ is $Ch^{p+1}$:

$$
v_{\text{full}} = u(t+k) + Ck^{p+1} + O(k^{p+2})
$$

$$
v_{\text{half}} = u(t+k) + 2C\left(\frac{k}{2}\right)^{p+1} + O(k^{p+2}) = u(t+k) + \frac{C k^{p+1}}{2^p} + O(k^{p+2})
$$

The factor of 2 in the half-step version comes from accumulating error over two steps.

**Error estimate.** Subtracting:

$$
v_{\text{half}} - v_{\text{full}} = Ck^{p+1}\left(\frac{1}{2^p} - 1\right) + O(k^{p+2})
$$

So $|v_{\text{half}} - v_{\text{full}}|$ estimates the local error, which drives the accept/reject decision and step-size adjustment.

**Extrapolation.** Since we know the error structure, we can cancel the leading error term:

$$
v_{\text{Rich}} = v_{\text{half}} + \frac{v_{\text{half}} - v_{\text{full}}}{2^p - 1}
$$

For $p = 1$: $v_{\text{Rich}} = 2v_{\text{half}} - v_{\text{full}}$, which gives an order $p+1 = 2$ accurate solution for free.

Richardson extrapolation does double duty: it provides an **error estimate** for adaptivity and **boosts the solution** from order 1 to order 2.

In [ ]:
from math import sqrt
from copy import copy

def be_adaptive(u0, f, *args, **kwargs):
    t0 = kwargs.pop('t0', 0.0)
    te = kwargs.pop('te', 1.0)
    rtol = kwargs.pop('rtol', 1e-4)
    atol = kwargs.pop('atol', 1e-4)
    a_min = kwargs.pop('a_min', 0.1)
    a_max = kwargs.pop('a_max', 5.0)
    safety = kwargs.pop('safety', 0.9)
    k_min = kwargs.pop('k_min', 1e-8)
    stepmax = kwargs.pop('stepmax', 10000)
    n_eqn = u0.size if isinstance(u0, np.ndarray) else 1  # Number of equations
    
    # Error messages
    msg = {0: 'Successful', 2: 'Step size dropped below kmin!', 3: 'Required more than stepmax steps!'}
    
    # Norm function to use
    norm = lambda x: np.max(np.abs(x))
    
    # Error code
    idid = 0
    
    # Time step setup
    k = kwargs.pop('k0', 1e-4 * (te - t0))
    
    # Current integrator state
    v = np.asarray(u0) # This always holds the current ODE value
    t = t0             # Current time
    
    # Logical variable to indicate if we can quit
    done = False
    
    # Some stats
    rejected_steps = 0
    accepted_steps = 0
    
    # Storage for output
    vs = []
    vs.append((t, np.copy(v)))  # Store initial condition
    
    # Newton method pars
    chi = 1e-3
    
    # Finally integrate
    while not done:
        if t + k >= te:
            k = te - t
            done = True
        else: # Make sure that the last time step isn't too small.
            k = min(k, 0.5 * (te - t))
        
        # Now compute new steps starting from (t, v)
        df = (f(t, v + delta) - f(t, v)) / delta
        
        # Code Newton method
        def newton_iter(v0, df, t, k):
            # Setup simplified Newton
            v_new = np.copy(v0)
            dv = 1.0
            done = False
            while not done:
                dv_new = (v0 + k * f(t, v_new) - v_new) / (1.0 - k * df)
                theta = abs(dv_new) / abs(dv)
                done = dv_new == 0.0 or abs(dv_new) <= chi * atol * (1 - theta) / theta
                dv = dv_new
                v_new += dv_new

            return v_new
        
        # Call first Newton iteration
        vf_new = newton_iter(v, df, t + k, k)      # Full step
        vh_new = newton_iter(v, df, t + 0.5 * k, 0.5 * k) # Half-step
        vh_new = newton_iter(vh_new, df, t + k, 0.5 * k) # Half-step
        
        # Estimate the error
        err = max(np.finfo(float).eps, norm(vf_new - vh_new) / (atol + max(norm(vh_new), norm(v)) * rtol))
        
        # Generate the new time-step
        knew = k * min(a_max, max(a_min, safety * sqrt(1./err)))
        
        # Make sure that new time-step is acceptable
        if k < k_min:
            idid = 2
            break
        
        if rejected_steps + accepted_steps > stepmax:
            idid = 3
            break
        
        if err <= 1.0: # Current error was small enough so update state
            # Now do the Richardson extrapolation step
            # v_new = vh_new + (vh_new - vf_new) / (2**p - 1)  p = 1 here
            v = 2. * vh_new - vf_new
            t = t + k
            
            # Store accepted step
            vs.append((t, np.copy(v)))
            
            # Update stats
            accepted_steps += 1
        else: # Error was too large -> repeat the step with new step-size
            done = False
            
            # Update stats
            rejected_steps += 1
        
        # Always use new step-size for next step
        k = knew

    print('Accepted steps = ', accepted_steps, ' Rejected steps = ', rejected_steps)
    print('{0:s}'.format(msg[idid]))
    return np.array(vs, dtype=[('t', 'float'), ('u', 'f8', (n_eqn, ))])


In [ ]:
a = -2.0

def f(t, u):
    return a * u

# Initial condition
u0 = 1.0

data = be_adaptive(u0, f, te=1.2, k0=0.2, atol=1e-3, rtol=1e-3)

In [ ]:
ts = data['t']
ue = u0 * np.exp(a * ts)
un = data['u'].flatten()
aerr = np.max(np.abs(ue - un))
rerr = np.max(np.abs((ue - un) / ue))

print('aerr = {0:.4e}; rerr = {1:.4e}'.format(aerr, rerr))

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].set_title('Number of approximation points = {0:d}'.format(ts.size))
#axs[0].plot(ts, np.abs(un - ue), marker='o', label='Numerical', color='k')
axs[0].plot(ts, un, label='Numerical', marker='o', color='k', ls='-')
axs[0].plot(ts, ue, label='Exact', color='r', ls='--')
axs[0].set_xlabel('$t$')
axs[0].set_ylabel('$u(t)$')
# axs[0].axhline(1e-3, color='r', ls='--')

axs[1].set_title('Step sizes')
axs[1].plot(ts[:-1], np.diff(ts), marker='o', label='Numerical', color='k')
axs[1].set_xlabel('$t$')
axs[1].set_ylabel('$k$')
fig.tight_layout()

# Backward Euler for Systems

We extend the adaptive backward Euler solver to **systems of ODEs** ($u \in \mathbb{R}^n$). The simplified Newton iteration now requires assembling and LU-factoring the Jacobian matrix $I - kJ_f$. We test on the **van der Pol oscillator**, a classic stiff system where adaptive step sizes are essential.

In [ ]:
from math import sqrt
from copy import copy

def ej(j, n):
    e = np.zeros(n, dtype=float)
    e[j] = 1.0
    return e

def be_adaptive(u0, f, *args, **kwargs):
    t0 = kwargs.pop('t0', 0.0)
    te = kwargs.pop('te', 1.0)
    rtol = kwargs.pop('rtol', 1e-4)
    atol = kwargs.pop('atol', 1e-4)
    a_min = kwargs.pop('a_min', 0.1)
    a_max = kwargs.pop('a_max', 2.0)
    safety = kwargs.pop('safety', 0.9)
    k_min = kwargs.pop('k_min', 1e-8)
    stepmax = kwargs.pop('stepmax', 10000)
    n_eqn = u0.size if isinstance(u0, np.ndarray) else 1  # Number of equations
    
    # Error messages
    msg = {0: 'Successful', 2: 'Step size dropped below kmin!', 3: 'Required more than stepmax steps!'}
    
    # Norm function to use
    norm = lambda x: np.max(np.abs(x))
    
    # Error code
    idid = 0
    
    # Time step setup
    k = kwargs.pop('k0', 1e-4 * (te - t0))
    
    # Current integrator state
    v = np.asarray(u0) # This always holds the current ODE value
    t = t0             # Current time
    
    # Logical variable to indicate if we can quit
    done = False
    
    # Some stats
    rejected_steps = 0
    r_newton_steps = 0
    accepted_steps = 0
    
    # Storage for output
    vs = []
    vs.append((t, np.copy(v)))  # Store initial condition
    
    # Newton method pars
    chi = 1e-1
    delta = sqrt(np.finfo(float).eps)
    
    # Finally integrate
    while not done:
        if t + k >= te:
            k = te - t
            done = True
        else: # Make sure that the last time step isn't too small.
            k = min(k, 0.5 * (te - t))

        # Now compute new steps starting from (t, v)
        df = np.zeros((n_eqn, n_eqn), dtype=float)
        fv = f(t, v)
        for i in range(n_eqn):
            ei = ej(i, n_eqn)
            df[:, i] = (f(t, v + ei * delta) - fv) / delta
            
        # Now compute LU decomposition of the matrix
        # lu, piv = LA.lu_factor(df)
        # We need to invert: (I - k A)
        # Now solve via:
        # x = LA.lu_solve((lu, piv), b)

        # Code Newton method
        def newton_iter(v0, df, t, k):
            # Setup simplified Newton
            v_new = np.copy(v0)
            done = False

            # Assemble the matrix and invert it: Can we do this better?
            T = np.eye(n_eqn) - k * df
            lu, piv = LA.lu_factor(T)

            # Do first iterate
            dv_new = LA.lu_solve((lu, piv), (v0 + k * f(t, v_new) - v_new))
            dv = dv_new
            v_new += dv_new

            # Quit if we are done now
            done = norm(dv) == 0.0

            while not done:
                # dv_new = (v0 + k * f(t, v_new) - v_new) / (1.0 - k * df)
                dv_new = LA.lu_solve((lu, piv), (v0 + k * f(t, v_new) - v_new))
                theta = norm(dv_new) / norm(dv)
                # print('theta = ', theta, ' error tol = ', chi * atol * (1 - theta) / theta)
                if theta > 1.0:
                    break

                done = norm(dv_new) == 0.0 or norm(dv_new) <= chi * atol * (1 - theta) / theta
                dv = dv_new
                v_new += dv_new

            return v_new, theta
        
        # Call first Newton iteration
        vf_new, theta = newton_iter(v,      df, t + k, k)      # Full step
        if theta > 1.:
            r_newton_steps += 1
            k = k * a_min
            continue
        
        vh_new, theta = newton_iter(v,      df, t + 0.5 * k, 0.5 * k) # Half-step
        if theta > 1.:
            r_newton_steps += 1
            k = k * a_min
            continue
            
        vh_new, theta = newton_iter(vh_new, df, t + k, 0.5 * k) # Half-step
        if theta > 1.:
            r_newton_steps += 1
            k = k * a_min
            continue
        
        # Estimate the error
        err = max(np.finfo(float).eps, norm(vf_new - vh_new) / (atol + max(norm(vh_new), norm(v)) * rtol))
        
        # Generate the new time-step
        knew = k * min(a_max, max(a_min, safety * sqrt(1./err)))
        
        # Make sure that new time-step is acceptable
        if k < k_min:
            idid = 2
            break
        
        if rejected_steps + accepted_steps > stepmax:
            idid = 3
            break
        
        if err <= 1.0: # Current error was small enough so update state
            # Now do the Richardson extrapolation step
            # v_new = vh_new + (vh_new - vf_new) / (2**p - 1)  p = 1 here
            v = 2. * vh_new - vf_new
            t = t + k
            
            # Store accepted step
            vs.append((t, np.copy(v)))
            
            # Update stats
            accepted_steps += 1
        else: # Error was too large -> repeat the step with new step-size
            done = False
            
            # Update stats
            rejected_steps += 1
        
        # Always use new step-size for next step
        k = knew

    print('Accepted steps = ', accepted_steps, 
          ' Rejected steps = ', rejected_steps,
          ' Rejected Newton steps = ', r_newton_steps)
    print('{0:s}'.format(msg[idid]))
    return np.array(vs, dtype=[('t', 'float'), ('u', 'f8', (n_eqn, ))])


In [ ]:
eps = 1e-2

def f(t, y):
    return np.array([
        y[1],
        ((1. - y[0]**2) * y[1] - y[0]) / eps
    ])

ic = np.array([2.0, 0.])

t0 = 0.0
te = 5.0

data1 = be_adaptive(ic, f, te=te, stepmax=400000, k0=0.1, atol=1e-2, rtol=1e-2)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(6.5, 5), sharex=True)

ts = data1['t']
un = data1['u']

ks = np.diff(ts)
ks = np.hstack((ks, ks[-1]))

axs[0].plot(ts, un[:, 0])
axs[0].set_ylabel('$u_1(t)$')
axs[1].plot(ts, un[:, 1])
axs[1].set_ylabel('$u_2(t)$')
axs[2].semilogy(ts, ks)
axs[2].set_ylabel('$k$')
axs[2].set_xlabel('Time $t$')
# axs[2].set_ylim([1e-7, 5e-3])
# axs[2].axhline(0.33 * eps, color='r', ls='--')
for ax in axs:
    ax.set_xlim([t0, te])

# Comparing Methods: Euler, RK2, Adams-Bashforth, PECE

We implement several one-step and multistep methods on the same test problem $u' = au$ and compare accuracy at fixed step size. Methods shown: forward Euler (order 1), RK2 midpoint (order 2), Adams-Bashforth 2-step (order 2), and predictor-corrector (PECE) variants. The plot overlays all methods against the exact solution.

In [ ]:
a = 2.0

def f(t, u):
    return a * u

# Initial condition
u0 = 1.0

# Time setup
Tf = 1.25
k  = 1./8  # Time step
numsteps = np.ceil(Tf / k).astype(int)
k = Tf / numsteps 
ts = np.arange(numsteps + 1) * k
ue = u0 * np.exp(a * ts)

# Create output vector
def fwd_euler(u0, numsteps):
    vs = np.empty(numsteps+1, dtype=float)
    vs[0] = u0
    
    v = u0 # This always holds the current ODE value
    for i in range(numsteps):
        t = k * (i - 1)
        v = v + k * f(t, v)
        vs[i+1] = v

    return vs

def rk2(u0, numsteps):
    vs = np.empty(numsteps+1, dtype=float)
    vs[0] = u0
    
    # Method constants
    a = 0.0 
    b = 1.0 
    alpha = 0.5 
    beta  = 0.5
    
    v = u0 # This always holds the current ODE value
    for i in range(numsteps):
        t  = k * (i - 1)
        r1 = f(t, v)
        r2 = f(t + alpha * k, v + beta * k * r1)
        v = v + k * (a * r1 + b * r2)
        vs[i+1] = v

    return vs

def adams_bashford(u0, numsteps):
    vs = np.empty(numsteps+1, dtype=float)
    vs[0] = u0

    # 1. Do one step of Euler
    vs[1] = u0 + k * f(0, u0)

    for i in range(1, numsteps):
        vs[i+1] = vs[i] + 1.5 * k * f(i * k, vs[i]) - 0.5 * k * f((i - 1) * k, vs[i-1])

    return vs


# Create output vector
def pece(u0, numpsteps):
    vs = np.empty(numsteps+1, dtype=float)
    vs[0] = u0
    
    v = u0 # This always holds the current ODE value
    for i in range(numsteps):
        t = k * (i - 1)
        # 1. Predictor
        ftv = f(t, v)
        y_star = v + k * ftv

        # 2. Correctors
        t_new = k * i
        v = v + 0.5 * k * (ftv + f(t_new, y_star))

        # 3. Store result
        vs[i+1] = v
        
    return vs


def adams_pece(u0, numsteps):
    vs = np.empty(numsteps+1, dtype=float)
    vs[0] = u0

    # 1. Do one step of PECE Euler
    y_star = u0 + k * f(0, u0)
    vs[1] = vs[0] + 0.5 * k * (f(0, u0) + f(k, y_star))

    for i in range(1, numsteps):
        y_star = vs[i] + 1.5 * k * f(i * k, vs[i]) - 0.5 * k * f((i - 1) * k, vs[i-1])
        vs[i+1] = vs[i] + 0.5 * k * (f((i + 1)*k, y_star) + f(i * k, vs[i]))

    return vs


# run some sims
v1s = fwd_euler(u0, numsteps)
e1s = np.max(np.abs(v1s - ue))

v5s = rk2(u0, numsteps)
e5s = np.max(np.abs(v5s - ue))

v2s = pece(u0, numsteps)
e2s = np.max(np.abs(v2s - ue))

v3s = adams_bashford(u0, numsteps)
e3s = np.max(np.abs(v3s - ue))

v4s = adams_pece(u0, numsteps)
e4s = np.max(np.abs(v4s - ue))

# Get timesteps 
tf = np.linspace(0, Tf, 1000)
fig, ax = plt.subplots(1, 1)
plt.plot(tf, np.exp(a * tf), lw=4, label='Exact solution')

# for i in range(1, numsteps+1):
#     C = vs[i] * np.exp(-a * ts[i])
#     plt.plot(tf, C * np.exp(a * tf), alpha=0.75)

ts = np.linspace(0, Tf, numsteps+1)
plt.plot(ts, v1s, color='k', lw=2, ls=':', zorder=4, marker='o', label='Forward Euler')
plt.plot(ts, v5s, color='k', lw=2, ls='-.', zorder=4, marker='o', label='RK2')
# plt.plot(ts, v2s, color='r', lw=2, zorder=5, marker='o', label='PECE Euler')
# plt.plot(ts, v3s, color='b', lw=2, ls=':', zorder=4, marker='o', label='Adams')
# plt.plot(ts, v4s, color='b', lw=2, zorder=5, marker='o', label='PECE Adams')

plt.title('Euler\'s method solution k = {0:.4f}'.format(k))
plt.ylabel('$y(t)$')
plt.xlabel('Time')
plt.legend(loc='best')
#plt.ylim([0, 100])
plt.xlim([0, Tf])

In [ ]:
print('Euler E1 = %.4g' % e1s)
print('PECE Euler E2 = %.4g' % e2s)
print('Adams E3 = %.4g' % e3s)
print('PECE Adams E4 = %.4g' % e4s)

# Adaptive Step-Size Control with RK2

We implement an adaptive RK2 (Heun) integrator using the **embedded pair** idea: the difference between a forward Euler step and an RK2 step estimates the local truncation error. The step size is adjusted so the estimated error stays below a user-specified tolerance. The plots show the solution and the adapted step sizes.

In [ ]:
from math import sqrt

def rk2_adaptive(u0, f, *args, **kwargs):
    t0 = kwargs.pop('t0', 0.0)
    te = kwargs.pop('te', 1.0)
    rtol = kwargs.pop('rtol', 1e-4)
    atol = kwargs.pop('atol', 1e-4)
    a_min = kwargs.pop('a_min', 0.1)
    a_max = kwargs.pop('a_max', 2.0)
    safety = kwargs.pop('safety', 0.9)
    k_min = kwargs.pop('k_min', 1e-8)
    stepmax = kwargs.pop('stepmax', 10000)
    n_eqn = u0.size if isinstance(u0, np.ndarray) else 1  # Number of equations
    
    # Error messages
    msg = {0: 'Success!', 2: 'Step size dropped below kmin!', 3: 'Required more than stepmax steps!'}
    
    # Error code
    idid = 0
    
    # Norm function to use
    norm = lambda x: np.max(np.abs(x))
    
    # Time step setup
    k = kwargs.pop('k0', 1e-4 * (te - t0))
    
    # Constants for the RK2 method
    a = 0.5
    b = 0.5
    alpha = 1.0 
    beta  = 1.0
    
    # Current integrator state
    v = np.asarray(u0) # This always holds the current ODE value
    t = t0             # Current time
    
    # Logical variable to indicate if we can quit
    done = False
    
    # Some stats
    rejected_steps = 0
    accepted_steps = 0
    
    # Storage for output
    vs = []
    vs.append((t, v))  # Store initial condition
    
    # Finally integrate
    while not done:
        if t + k >= te:
            k = te - t
            done = True
        else: # Make sure that the last time step isn't too small.
            k = min(k, 0.5 * (te - t))
        
        # Now compute new steps starting from (t, v)
        # RK2: Compute the two slopes
        r1 = f(t, v)
        r2 = f(t + alpha * k, v + beta * k * r1)
        
        # Compute Euler step
        w = v + k * r1
        
        # Compute RK2 step
        vnew = v + k * (a * r1 + b * r2)
        
        # Estimate the error
        err = max(np.finfo(float).eps, norm(w - vnew) / (atol + max(norm(vnew), norm(v)) * rtol))
        
        # Generate the new time-step
        knew = k * min(a_max, max(a_min, safety * sqrt(1./err)))
        
        # Make sure that new time-step is acceptable
        if k < k_min:
            idid = 2
            break
            
        if accepted_steps + rejected_steps > stepmax:
            idid = 3
            break
        
        if err <= 1.0: # Current error was small enough so update state
            v = vnew
            t = t + k
            
            # Store accepted step
            vs.append((t, v))
            
            # Update stats
            accepted_steps += 1
        else: # Error was too large -> repeat the step with new step-size
            done = False
            
            # Update stats
            rejected_steps += 1
        
        # Always use new step-size for next step
        k = knew

    print('{0:s}'.format(msg[idid]))
    print('Accepted steps = ', accepted_steps, ' Rejected steps = ', rejected_steps)
    return np.array(vs, dtype=[('t', 'float'), ('u', 'f8', (n_eqn, ))])


In [ ]:
a = 2.0

def f(t, u):
    return a * u

# Initial condition
u0 = 1.0

te = 1.15
data = rk2_adaptive(u0, f, te=te, k0=0.1, atol=1e-2, rtol=1e-2)

In [ ]:
ts = data['t']
ue = u0 * np.exp(a * ts)
un = data['u'].flatten()
aerr = np.max(np.abs(ue - un))
rerr = np.max(np.abs((ue - un) / ue))

print('aerr = {0:.4e}; rerr = {1:.4e}'.format(aerr, rerr))

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].set_title('Number of approximation points = {0:d}'.format(ts.size))
axs[0].plot(ts, un, marker='o', label='Numerical', color='k')
axs[0].plot(ts, ue, label='Exact', color='r', ls='--')
axs[0].set_xlabel('$t$')
axs[0].set_ylabel('$u(t)$')
axs[0].axvline(te, color='r', ls='--')

axs[1].set_title('Step sizes')
axs[1].plot(ts[:-1], np.diff(ts), marker='o', label='Numerical', color='k')
axs[1].set_xlabel('$t$')
axs[1].set_ylabel('$k$')
fig.tight_layout()

# Adaptive Predictor-Corrector

An adaptive **PECE** (Predict-Evaluate-Correct-Evaluate) integrator: the predictor is a forward Euler step, the corrector applies the trapezoidal rule. Their difference estimates the local error and drives step-size control. Tested on $u' = au$ with decaying solution.

In [ ]:
from math import sqrt

def pc_adaptive(u0, f, *args, **kwargs):
    t0 = kwargs.pop('t0', 0.0)
    te = kwargs.pop('te', 1.0)
    rtol = kwargs.pop('rtol', 1e-4)
    atol = kwargs.pop('atol', 1e-4)
    a_min = kwargs.pop('a_min', 0.1)
    a_max = kwargs.pop('a_max', 5.0)
    safety = kwargs.pop('safety', 0.9)
    k_min = kwargs.pop('k_min', 1e-8)
    stepmax = kwargs.pop('stepmax', 10000)
    n_eqn = u0.size if isinstance(u0, np.ndarray) else 1  # Number of equations
    
    # Error messages
    msg = {0: 'Successful', 2: 'Step size dropped below kmin!', 3: 'Required more than stepmax steps!'}
    
    # Norm function to use
    norm = lambda x: np.max(np.abs(x))
    
    # Error code
    idid = 0
    
    # Time step setup
    k = kwargs.pop('k0', 1e-4 * (te - t0))
    
    # Current integrator state
    v = np.asarray(u0) # This always holds the current ODE value
    t = t0             # Current time
    
    # Logical variable to indicate if we can quit
    done = False
    
    # Some stats
    rejected_steps = 0
    accepted_steps = 0
    
    # Storage for output
    vs = []
    vs.append((t, v))  # Store initial condition
    
    # Finally integrate
    while not done:
        if t + k >= te:
            k = te - t
            done = True
        else: # Make sure that the last time step isn't too small.
            k = min(k, 0.5 * (te - t))
        
        # Now compute new steps starting from (t, v)
        # RK2: Compute the two slopes
        r1 = f(t, v)
        
        # Compute Euler step i.e. the predictors
        w = v + k * r1
        
        # Apply the corrector i.e. trapezoidal rule
        vnew = v + 0.5 * k * (r1 + f(t + k, w))
        
        # Estimate the error
        err = max(np.finfo(float).eps, norm(w - vnew) / (atol + max(norm(vnew), norm(v)) * rtol))
        
        # Generate the new time-step
        knew = k * min(a_max, max(a_min, safety * sqrt(1./err)))
        
        # Make sure that new time-step is acceptable
        if k < k_min:
            idid = 2
            break
        
        if rejected_steps + accepted_steps > stepmax:
            idid = 3
            break
        
        if err <= 1.0: # Current error was small enough so update state
            v = vnew
            t = t + k
            
            # Store accepted step
            vs.append((t, v))
            
            # Update stats
            accepted_steps += 1
        else: # Error was too large -> repeat the step with new step-size
            done = False
            
            # Update stats
            rejected_steps += 1
        
        # Always use new step-size for next step
        k = knew

    print('Accepted steps = ', accepted_steps, ' Rejected steps = ', rejected_steps)
    print('{0:s}'.format(msg[idid]))
    return np.array(vs, dtype=[('t', 'float'), ('u', 'f8', (n_eqn, ))])


In [ ]:
a = -2.0

def f(t, u):
    return a * u

# Initial condition
u0 = 1.0

data = pc_adaptive(u0, f, te=1.2, k0=0.1, atol=1e-2, rtol=1e-2)

In [ ]:
ts = data['t']
ue = u0 * np.exp(a * ts)
un = data['u'].flatten()
aerr = np.max(np.abs(ue - un))
rerr = np.max(np.abs((ue - un) / ue))

print('aerr = {0:.4e}; rerr = {1:.4e}'.format(aerr, rerr))

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].set_title('Number of approximation points = {0:d}'.format(ts.size))
#axs[0].plot(ts, un, marker='o', label='Numerical', color='k')
axs[0].plot(ts, np.abs(un - ue), marker='o', label='Numerical', color='k')
#axs[0].plot(ts, ue, label='Exact', color='r', ls='--')
axs[0].axhline(1e-2, color='r', ls='--')
axs[0].set_xlabel('$t$')
axs[0].set_ylabel('$u(t)$')

axs[1].set_title('Step sizes')
axs[1].plot(ts[:-1], np.diff(ts), marker='o', label='Numerical', color='k')
axs[1].set_xlabel('$t$')
axs[1].set_ylabel('$k$')
fig.tight_layout()

# Van der Pol Oscillator (Stiff System)

The van der Pol oscillator with small $\varepsilon$ is a classic **stiff** problem: the solution has sharp transients separated by slow phases. The adaptive RK2 integrator must take very small steps during transients and can relax during slow phases. The bottom panel shows how the step size adapts automatically.

In [ ]:
eps = 1e-2

def f(t, y):
    return np.array([
        y[1],
        ((1. - y[0]**2) * y[1] - y[0]) / eps
    ])
    

ic = np.array([2.0, 0.])

t0 = 0.0
te = 5.0

data1 = rk2_adaptive(ic, f, te=te, stepmax=400000, k0=0.1, atol=1e-2, rtol=1e-2)
# data2 = pc_adaptive(ic, f, te=te, stepmax=40000, k0=0.1, atol=1e-4, rtol=1e-4)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(6.5, 5), sharex=True)

ts = data1['t']
un = data1['u']

ks = np.diff(ts)
ks = np.hstack((ks, ks[-1]))

axs[0].plot(ts, un[:, 0])
axs[0].set_ylabel('$u_1(t)$')
axs[1].plot(ts, un[:, 1])
axs[1].set_ylabel('$u_2(t)$')
axs[2].semilogy(ts, ks)
axs[2].set_ylabel('$k$')
axs[2].set_xlabel('Time $t$')
# axs[2].set_ylim([1e-7, 5e-3])
# axs[2].axhline(0.33 * eps, color='r', ls='--')
for ax in axs:
    ax.set_xlim([t0, te])

# Stability of Forward Euler

We solve $u' = \lambda(u - \cos t) - \sin t$, which has exact solution $u(t) = \cos t$, for three values of $\lambda$. With fixed step size $h = 10^{-3}$: $\lambda = 0$ and $\lambda = -10$ are stable, but $\lambda = -2100$ violates the stability condition $h < 2/|\lambda| \approx 9.5 \times 10^{-4}$ and the solution blows up.

In [ ]:
def f1(t, u):
    return -np.sin(t)

# setup problem
u0 = 1.0
t0 = 0.0
te = 2.0

# Create output vector
def fwd_euler(u0, f, k, numsteps):
    vs = np.empty(numsteps+1, dtype=float)
    vs[0] = u0
    
    v = u0 # This always holds the current ODE value
    for i in range(numsteps):
        t = k * i
        v = v + k * f(t, v)
        vs[i+1] = v

    return vs

# First goal with k = 1e-3
k  = 1e-3
numsteps = np.ceil(te / k).astype(int)
k = te / numsteps 
ts = np.arange(numsteps + 1) * k
ue = np.cos(ts)

# Execute Euler's method
vs1 = fwd_euler(u0, f1, k, numsteps)
err = np.abs(vs1 - ue)[-1]  # Grab last error

print(f'lambda = {0.0:10.1f}, h = {k:.6e}, error = {err:.6e}')

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].plot(ts, vs1)
axs[0].plot(ts, ue, ls='--')
axs[0].set_title(r'$\lambda = 0.0$')

# First with lambda
lam = -10.0

def f2(t, u):
    return lam * (u - np.cos(t)) - np.sin(t)

# Solve
k  = 1e-3
numsteps = np.ceil(te / k).astype(int)
k = te / numsteps 
ts = np.arange(numsteps + 1) * k
ue = np.cos(ts)

# Execute Euler's method
vs2 = fwd_euler(u0, f2, k, numsteps)
err = np.abs(vs2 - ue)[-1]  # Grab last error

print(f'lambda = {lam:10.1f}, h = {k:.6e}, error = {err:.6e}')

axs[1].plot(ts, vs2)
axs[1].plot(ts, ue, ls='--')
axs[1].set_title(r'$\lambda = -10.0$')

# Let's do it again 
lam = -2100.0

def f2(t, u):
    return lam * (u - np.cos(t)) - np.sin(t)

# Solve
k  = 1e-3
numsteps = np.ceil(te / k).astype(int)
k = te / numsteps 
ts = np.arange(numsteps + 1) * k
ue = np.cos(ts)

# Execute Euler's method
vs3 = fwd_euler(u0, f2, k, numsteps)
err = np.abs(vs3 - ue)[-1]  # Grab last error

print(f'lambda = {lam:10.1f}, h = {k:.6e}, error = {err:.6e}')

axs[2].plot(ts, vs3)
axs[2].plot(ts, ue, ls='--')
axs[2].set_title(r'$\lambda = -2100.0$')

## Rescuing the unstable case

For $\lambda = -2100$, we reduce $h$ until we satisfy the stability condition $h|\lambda| \leq 2$. The output shows the error dropping dramatically once $h$ is small enough.

In [ ]:
# Let's do it again 
lam = -2100.0

def f2(t, u):
    return lam * (u - np.cos(t)) - np.sin(t)

# Solve
for k in [0.000976, 0.000950, 0.0008, 0.0004]:
    numsteps = np.ceil(te / k).astype(int)
    k = te / numsteps 
    ts = np.arange(numsteps + 1) * k
    ue = np.cos(ts)
    
    # Execute Euler's method
    vs3 = fwd_euler(u0, f2, k, numsteps)
    err = np.abs(vs3 - ue)[-1]  # Grab last error
    
    print(f'lambda = {lam:10.1f}, h = {k:.2e}, error = {err:.6e}')


# Stiffness: Forward vs Backward Euler

We add a stiff term $-100(u - \cos t)$ to the ODE and compare forward and backward Euler across a range of step sizes. Forward Euler requires $h < 2/100 = 0.02$ for stability; for larger $h$ the error explodes. Backward Euler is unconditionally stable and converges at the expected $O(h)$ rate regardless of step size.

In [ ]:
logkset = np.arange(1, 10)

tmax = 2.
exact = np.cos(tmax)
ex_errs = []

for logk in logkset:
    k = 0.5**logk
    nsteps = int(tmax/k)
    t = 0
    v = np.cos(t)
    f = -np.sin(t)
    for i in range(nsteps):
        t = i * k
        # f = -np.sin(t)
        f = -np.sin(t) - 100 * (v - np.cos(t))
        vnew = v + k * f
        v = vnew
    
    error = np.abs(v - exact)
    ex_errs.append(error)

plt.ylabel('Error')
plt.xlabel('Time step $k$')
plt.loglog(0.5**logkset, ex_errs, marker='o')
plt.grid()

In [ ]:
logkset = np.arange(1, 10)

tmax = 2.
exact = np.cos(tmax)
errs = []

for logk in logkset:
    k = 0.5**logk
    nsteps = int(tmax/k)
    t = 0
    v = np.cos(t)
    f = -np.sin(t)
    for i in range(nsteps):
        t = i * k
        # First
        vnew = v + k * (-np.sin(t + k))
        # Second
        # vnew = (v + k * (-np.sin(k + t) + 100*np.cos(t + k))) / (1 + k * 100)
        v = vnew
    
    error = np.abs(v - exact)
    errs.append(error)

plt.ylabel('Error')
plt.xlabel('Time step $k$')
plt.loglog(0.5**logkset, errs, marker='o', label='Bwd Euler')
plt.loglog(0.5**logkset, ex_errs, marker='o', label='Fwd Euler')
plt.grid()
plt.legend(loc='best')